# LEA School District Finance Survey – Regional Level Analysis

#### Note: This project is still in progress

In [1]:
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from utilities import Utilities

In [2]:
# Create Instance of Utilities (A class with various data operations that will be used throughout the project)
utils = Utilities()

# Establish Database Connection
# The upcoming data retrieval method will switch to CSV format if the database connection fails
conn = utils.create_database_conn()

#### Note on Database Connectivity and Data Analysis

This notebook facilitates a fixed analysis of static data covering the years 2010 to 2020, designed to ensure consistency and replicability of results across different environments. The choice between data extraction methods is streamlined for efficiency and convenience:

- **Direct Database Connection**: The notebook attempts to connect to the database at the outset using provided credentials. A successful connection is preferred, especially for conducting the analysis, as it allows direct access to the database. This enables flexibility in querying additional data or making structural adjustments without immediate concern for updating CSV files. The data in the database is static, mirroring the fixed analysis period, and matches exactly the data exported into the CSV files.

- **CSV File Access as Fallback**: In cases where the database connection cannot be established (due to incorrect credentials, network access issues, or database unavailability), the notebook automatically switches to using pre-exported CSV files (`student_counts_by_state.csv` and `tech_vocational_expenditures_2010_to_2020.csv`). These files are generated using the same queries as those within the notebook, ensuring that the data and subsequent analysis results will match those obtained directly from the database. This method guarantees that the analysis can be consistently replicated, regardless of the environment setup.

The transition between the database and CSV files is handled automatically, eliminating the need for manual configuration. This approach ensures that, whether accessing the database directly or using CSV files, the analysis remains consistent and true to the static dataset from 2010 to 2020.

## Expenditure Analysis

In [3]:
# Query expenditure data
query = """
SELECT 
    state,
    region,
    expenditure_title, 
    year, 
    SUM(amount) AS amount, 
    AVG(amount_z_score) AS amount_z_score_avg 
FROM expenses.expenditure_zscores_by_state_year
WHERE expenditure_title ILIKE '%tech%' 
OR expenditure_title ILIKE '%vocation%'
GROUP BY state, region, year, expenditure_title
ORDER BY state, year, expenditure_title;
"""

In [4]:
if conn:
    tech_vocational_expenditures = utils.execute_sql(query, conn)
else:
        try:
            tech_vocational_expenditures = pd.read_csv(os.path.join('..','tech_vocational_expenditures_2010_to_2020.csv'), 
                                                       dtype={'amount_z_score_avg': 'str'})
        except FileNotFoundError:
            print("CSV file 'tech_vocational_expenditures_2010_to_2020.csv' not found. Please ensure the file is in the correct directory.")
            raise 
        except Exception as e:
            print(f"An error occurred while loading the CSV file: {e}")
            raise 
tech_vocational_expenditures.head()

,state,region,expenditure_title,year,amount,amount_z_score_avg
0,Alabama,South,teacher_salaries_vocational_education,2010.0,110309000.0,-1.67803030303030303E-19
1,Alabama,South,teacher_salaries_vocational_education,2011.0,107614000.0,-6.14166666666666667E-19
2,Alabama,South,teacher_salaries_vocational_education,2012.0,105162000.0,-2.25606060606060606E-19
3,Alabama,South,teacher_salaries_vocational_education,2013.0,108247000.0,1.71343283582089552E-19
4,Alabama,South,teacher_salaries_vocational_education,2014.0,110787000.0,6.56666666666666667E-19


In [5]:
tech_vocational_expenditures.dtypes

state                  object
region                 object
expenditure_title      object
year                  float64
amount                 object
amount_z_score_avg     object
dtype: object

In [6]:
tech_vocational_expenditures = tech_vocational_expenditures.astype({'year' : int,
                                                                    'amount' : 'float64',
                                                                    'amount_z_score_avg' : 'float64'
                                                                    })

tech_vocational_expenditures.dtypes

state                  object
region                 object
expenditure_title      object
year                    int64
amount                float64
amount_z_score_avg    float64
dtype: object

In [7]:
tech_vocational_expenditures.head()

,state,region,expenditure_title,year,amount,amount_z_score_avg
0,Alabama,South,teacher_salaries_vocational_education,2010,110309000.0,-1.678030e-19
1,Alabama,South,teacher_salaries_vocational_education,2011,107614000.0,-6.141667e-19
2,Alabama,South,teacher_salaries_vocational_education,2012,105162000.0,-2.256061e-19
3,Alabama,South,teacher_salaries_vocational_education,2013,108247000.0,1.713433e-19
4,Alabama,South,teacher_salaries_vocational_education,2014,110787000.0,6.566667e-19


In [8]:
query = """
SELECT 
	e.state,
    e.region, 
	DATE_PART('year', stats.year) AS year, 
	SUM(stats.fall_membership) AS student_count
FROM entity.annual_stats AS stats
INNER JOIN entity.entity as e
	ON stats.census_id = e.census_id
GROUP BY e.state, e.region, stats.year
ORDER BY e.state, stats.year;
"""

In [9]:
if conn:
    student_counts_by_state_year = utils.execute_sql(query, conn)
else:
    try:
        student_counts_by_state_year = pd.read_csv(os.path.join('..','student_counts_by_state.csv'))
    except FileNotFoundError:
        print("CSV file 'student_counts_by_state.csv' not found. Please ensure the file is in the correct directory.")
        raise  # Reraise the exception to stop the notebook
    except Exception as e:
        print(f"An error occurred while loading the CSV file: {e}")
        raise  # Reraise the exception to stop the notebook

# Display the head of the DataFrame
student_counts_by_state_year.head()

,state,region,year,student_count
0,Alabama,South,2010.0,748889.0
1,Alabama,South,2011.0,738322.0
2,Alabama,South,2012.0,744621.0
3,Alabama,South,2013.0,744548.0
4,Alabama,South,2014.0,740081.0


In [10]:
student_counts_by_state_year['year'] = student_counts_by_state_year['year'].astype(int)
student_counts_by_state_year['student_count'] = student_counts_by_state_year['student_count'].astype(int)

student_counts_by_state_year.dtypes

state            object
region           object
year              int64
student_count     int64
dtype: object

In [11]:
regionwise_tech_vocational_expenditures = tech_vocational_expenditures.groupby(['region', 'year', 'expenditure_title'])['amount'].sum().reset_index()
regionwise_tech_vocational_expenditures.head()

,region,year,expenditure_title,amount
0,Midwest,2010,teacher_salaries_vocational_education,894448000.0
1,Midwest,2011,teacher_salaries_vocational_education,954296000.0
2,Midwest,2012,teacher_salaries_vocational_education,918036000.0
3,Midwest,2013,teacher_salaries_vocational_education,906328000.0
4,Midwest,2014,teacher_salaries_vocational_education,891829000.0


In [12]:
student_counts_by_region_year = student_counts_by_state_year.groupby(['region', 'year'])['student_count'].sum().reset_index()

student_counts_by_region_year.head()

,region,year,student_count
0,Midwest,2010,10348108
1,Midwest,2011,10276081
2,Midwest,2012,10203489
3,Midwest,2013,10133587
4,Midwest,2014,10141778


In [13]:
regionwise_tech_vocational_expenditures = regionwise_tech_vocational_expenditures.merge(student_counts_by_region_year, how='inner', on= ['region', 'year'], sort=True)
regionwise_tech_vocational_expenditures.head()

,region,year,expenditure_title,amount,student_count
0,Midwest,2010,teacher_salaries_vocational_education,894448000.0,10348108
1,Midwest,2011,teacher_salaries_vocational_education,954296000.0,10276081
2,Midwest,2012,teacher_salaries_vocational_education,918036000.0,10203489
3,Midwest,2013,teacher_salaries_vocational_education,906328000.0,10133587
4,Midwest,2014,teacher_salaries_vocational_education,891829000.0,10141778


In [14]:
regionwise_tech_vocational_expenditures['cost_per_student'] = regionwise_tech_vocational_expenditures['amount'] / regionwise_tech_vocational_expenditures['student_count']

regionwise_tech_vocational_expenditures.head()

,region,year,expenditure_title,amount,student_count,cost_per_student
0,Midwest,2010,teacher_salaries_vocational_education,894448000.0,10348108,86.435897
1,Midwest,2011,teacher_salaries_vocational_education,954296000.0,10276081,92.865753
2,Midwest,2012,teacher_salaries_vocational_education,918036000.0,10203489,89.972753
3,Midwest,2013,teacher_salaries_vocational_education,906328000.0,10133587,89.438024
4,Midwest,2014,teacher_salaries_vocational_education,891829000.0,10141778,87.936159


### Understanding "Cost Per Student" Calculation

**Methodology:**

For the purpose of this analysis, the "cost per student" metric is calculated as the total expenditure amount divided by the number of students for the given year. The formula used is as follows:

```python
regionwise_data_w_student_counts['cost_per_student'] = regionwise_data_w_student_counts['amount'] / regionwise_data_w_student_counts['student_count']
```

This calculation aims to provide an approximate value indicating the expenditure per student based on the available data of total expenditure and student counts.

**Limitations:**

It's essential to acknowledge the limitations of this calculation. The derived "cost per student" metric does not account for the nuanced breakdown of expenses directly associated with each student. The analysis lacks the granularity to determine what percentage of funds is genuinely allocated towards the student's educational expenses versus other operational or administrative costs. As a result, the calculated "cost per student" serves as an overall indicator and may not accurately represent the true cost of education per student.

In [15]:
regionwise_expenditure_totals = regionwise_tech_vocational_expenditures.groupby(['region', 'year'])['amount'].sum().reset_index()
regionwise_expenditure_totals.sort_values(['region', 'year'], inplace= True)
regionwise_expenditure_totals.head()

,region,year,amount
0,Midwest,2010,894448000.0
1,Midwest,2011,954296000.0
2,Midwest,2012,918036000.0
3,Midwest,2013,906328000.0
4,Midwest,2014,891829000.0


In [16]:
regionwise_expenditure_totals = regionwise_expenditure_totals.sort_values(by=['region', 'year'])

# Calculate the year-to-year difference and growth rate
regionwise_expenditure_totals['yearly_difference'] = regionwise_expenditure_totals.groupby('region')['amount'].diff()
regionwise_expenditure_totals['growth_rate'] = regionwise_expenditure_totals.groupby('region')['amount'].pct_change() * 100

regionwise_expenditure_totals.head()

,region,year,amount,yearly_difference,growth_rate
0,Midwest,2010,894448000.0,NaN,NaN
1,Midwest,2011,954296000.0,59848000.0,6.691054
2,Midwest,2012,918036000.0,-36260000.0,-3.799660
3,Midwest,2013,906328000.0,-11708000.0,-1.275331
4,Midwest,2014,891829000.0,-14499000.0,-1.599752


In [17]:
national_totals_line = px.line(regionwise_expenditure_totals.groupby('year')['amount'].sum().reset_index(),
                                 x= 'year',
                                 y= 'amount',
                                 title= 'Tech and Vocational Education Expenditures Over Time')

In [18]:
regionwise_totals_line = px.line(regionwise_expenditure_totals,
                                 x= 'year',
                                 y= 'amount',
                                 color= 'region',
                                 title= 'Tech and Vocational Education Expenditures by Region Over Time',
                                 hover_data={'yearly_difference': ':.2f',
                                             'growth_rate' : ':.2f'})

#### Midwest Calculations

In [19]:
MIDWEST_TOTAL_2010 = regionwise_expenditure_totals.loc[(regionwise_expenditure_totals['region'] == 'Midwest') & (regionwise_expenditure_totals['year'] == 2010), 'amount'].values[0]
MIDWEST_TOTAL_2015 = regionwise_expenditure_totals.loc[(regionwise_expenditure_totals['region'] == 'Midwest') & (regionwise_expenditure_totals['year'] == 2015), 'amount'].values[0]
MIDWEST_TOTAL_2016 = regionwise_expenditure_totals.loc[(regionwise_expenditure_totals['region'] == 'Midwest') & (regionwise_expenditure_totals['year'] == 2016), 'amount'].values[0]
MIDWEST_TOTAL_2019 = regionwise_expenditure_totals.loc[(regionwise_expenditure_totals['region'] == 'Midwest') & (regionwise_expenditure_totals['year'] == 2019), 'amount'].values[0]

MIDWEST_DIFF_2010_2015 = MIDWEST_TOTAL_2015 - MIDWEST_TOTAL_2010
MIDWEST_DIFF_2016_2019 = MIDWEST_TOTAL_2019 - MIDWEST_TOTAL_2016

MIDWEST_PERCENT_DIFF_2010_2015 = utils.calculate_percentage_difference(MIDWEST_TOTAL_2010, MIDWEST_TOTAL_2015)
MIDWEST_MEAN_GROWTH_2010_2015 = utils.calculate_mean_growth_rate(regionwise_expenditure_totals, start_year=2010, end_year=2015, region= 'Midwest')
MIDWEST_MEAN_GROWTH_2016_2019 = utils.calculate_mean_growth_rate(regionwise_expenditure_totals, start_year=2016, end_year=2019, region= 'Midwest')


print(f'The Midwest had minimal growth between 2010 and 2015, resulting in a {MIDWEST_PERCENT_DIFF_2010_2015:.2f}% increase and an average growth rate of {MIDWEST_MEAN_GROWTH_2010_2015:.2f}%. This totaled in a ${MIDWEST_DIFF_2010_2015} difference.')
print(f'2016 to 2019 saw a mean growth rate of {MIDWEST_MEAN_GROWTH_2016_2019:.2f}%, resulting in a ${MIDWEST_DIFF_2016_2019} increase.')

The Midwest had minimal growth between 2010 and 2015, resulting in a 14.83% increase and an average growth rate of 3.04%. This totaled in a $132691000.0 difference.
2016 to 2019 saw a mean growth rate of 32.87%, resulting in a $959967000.0 increase.


#### Northeast Calculations

In [20]:
NE_TOTAL_2010 = regionwise_expenditure_totals.loc[(regionwise_expenditure_totals['region'] == 'Northeast') & (regionwise_expenditure_totals['year'] == 2010), 'amount'].values[0]
NE_TOTAL_2013 = regionwise_expenditure_totals.loc[(regionwise_expenditure_totals['region'] == 'Northeast') & (regionwise_expenditure_totals['year'] == 2013), 'amount'].values[0]
NE_TOTAL_2015 = regionwise_expenditure_totals.loc[(regionwise_expenditure_totals['region'] == 'Northeast') & (regionwise_expenditure_totals['year'] == 2015), 'amount'].values[0]
NE_TOTAL_2020 = regionwise_expenditure_totals.loc[(regionwise_expenditure_totals['region'] == 'Northeast') & (regionwise_expenditure_totals['year'] == 2020), 'amount'].values[0]

NE_DIFF_2010_2013 = NE_TOTAL_2013 - NE_TOTAL_2010
NE_DIFF_2013_2015 = NE_TOTAL_2015 - NE_TOTAL_2013
NE_DIFF_2015_2020 = NE_TOTAL_2020 - NE_TOTAL_2015

NE_MEAN_GROWTH_2010_2013 = utils.calculate_mean_growth_rate(regionwise_expenditure_totals, start_year=2010, end_year=2013, region= 'Northeast')
NE_MEAN_GROWTH_2013_2015 = utils.calculate_mean_growth_rate(regionwise_expenditure_totals, start_year=2013, end_year=2015, region= 'Northeast')
NE_MEAN_GROWTH_2015_2020 = utils.calculate_mean_growth_rate(regionwise_expenditure_totals, start_year=2015, end_year=2020, region= 'Northeast')

print(f'The Northeast had minimal growth between 2010 and 2013, resulting in a mean growth rate of {NE_MEAN_GROWTH_2010_2013:.2f}% and a total difference of ${NE_DIFF_2010_2013}.')
print(f'They saw their first spike in expenditures in 2015. Between 2013 and 2015, their mean growth rate increased to {NE_MEAN_GROWTH_2013_2015:.2f}%.')
print(f'After this initial spike, their growth maintained a fairly constant rate from 2015 to 2020, at a mean growth rate of {NE_MEAN_GROWTH_2015_2020:.2f}% and a total difference of ${NE_DIFF_2015_2020}.')

The Northeast had minimal growth between 2010 and 2013, resulting in a mean growth rate of 0.71% and a total difference of $23137000.0.
They saw their first spike in expenditures in 2015. Between 2013 and 2015, their mean growth rate increased to 6.58%.
After this initial spike, their growth maintained a fairly constant rate from 2015 to 2020, at a mean growth rate of 11.03% and a total difference of $1162070000.0.


#### South Calculations

In [21]:
SOUTH_TOTAL_2014 = regionwise_expenditure_totals.loc[(regionwise_expenditure_totals['region'] == 'South') & (regionwise_expenditure_totals['year'] == 2014), 'amount'].values[0]
SOUTH_TOTAL_2016 = regionwise_expenditure_totals.loc[(regionwise_expenditure_totals['region'] == 'South') & (regionwise_expenditure_totals['year'] == 2016), 'amount'].values[0]
SOUTH_TOTAL_2017 = regionwise_expenditure_totals.loc[(regionwise_expenditure_totals['region'] == 'South') & (regionwise_expenditure_totals['year'] == 2017), 'amount'].values[0]
SOUTH_TOTAL_2020 = regionwise_expenditure_totals.loc[(regionwise_expenditure_totals['region'] == 'South') & (regionwise_expenditure_totals['year'] == 2020), 'amount'].values[0]

SOUTH_DIFF_2014_2016 = SOUTH_TOTAL_2016 - SOUTH_TOTAL_2014
SOUTH_DIFF_2017_2020 = SOUTH_TOTAL_2020 - SOUTH_TOTAL_2017

SOUTH_PERCENT_DIFF_2014_2016 = utils.calculate_percentage_difference(SOUTH_TOTAL_2014, SOUTH_TOTAL_2016)
SOUTH_MEAN_GROWTH_2018_2020 = utils.calculate_mean_growth_rate(regionwise_expenditure_totals, start_year=2017, end_year=2020, region= 'South')

print(f'The South saw its greatest spike in spending betweeen 2014 and 2016, resulting in a {SOUTH_PERCENT_DIFF_2014_2016:.2f}% increase, with a total difference of ${SOUTH_DIFF_2014_2016}.')
print(f'2017 to 2020 saw a mean growth rate of {SOUTH_MEAN_GROWTH_2018_2020:.2f}%, resulting in a ${SOUTH_DIFF_2017_2020} increase.')

The South saw its greatest spike in spending betweeen 2014 and 2016, resulting in a 99.71% increase, with a total difference of $2326078000.0.
2017 to 2020 saw a mean growth rate of 10.09%, resulting in a $1987658000.0 increase.


#### West Calculations

In [22]:
WEST_TOTAL_2010 = regionwise_expenditure_totals.loc[(regionwise_expenditure_totals['region'] == 'West') & (regionwise_expenditure_totals['year'] == 2010), 'amount'].values[0]
WEST_TOTAL_2014 = regionwise_expenditure_totals.loc[(regionwise_expenditure_totals['region'] == 'West') & (regionwise_expenditure_totals['year'] == 2014), 'amount'].values[0]
WEST_TOTAL_2017 = regionwise_expenditure_totals.loc[(regionwise_expenditure_totals['region'] == 'West') & (regionwise_expenditure_totals['year'] == 2017), 'amount'].values[0]
WEST_TOTAL_2018 = regionwise_expenditure_totals.loc[(regionwise_expenditure_totals['region'] == 'West') & (regionwise_expenditure_totals['year'] == 2018), 'amount'].values[0]
WEST_TOTAL_2020 = regionwise_expenditure_totals.loc[(regionwise_expenditure_totals['region'] == 'West') & (regionwise_expenditure_totals['year'] == 2020), 'amount'].values[0]

WEST_DIFF_2010_2014 = WEST_TOTAL_2014 - WEST_TOTAL_2010
WEST_DIFF_2014_2017 = WEST_TOTAL_2017 - WEST_TOTAL_2014
WEST_DIFF_2018_2020 = WEST_TOTAL_2020 - WEST_TOTAL_2018

WEST_PERCENT_DIFF_2014_2017 = utils.calculate_percentage_difference(WEST_TOTAL_2014, WEST_TOTAL_2017)
WEST_PERCENT_DIFF_2018_2020 = utils.calculate_percentage_difference(WEST_TOTAL_2018, WEST_TOTAL_2020)
WEST_MEAN_GROWTH_2010_2014 = utils.calculate_mean_growth_rate(regionwise_expenditure_totals, start_year=2010, end_year=2014, region= 'West')
WEST_MEAN_GROWTH_2018_2020 = utils.calculate_mean_growth_rate(regionwise_expenditure_totals, start_year=2018, end_year=2020, region= 'West')

print(f'Between 2010 and 2014, the West had minimal growth in expenditures. This resulted in a mean growth rate of {WEST_MEAN_GROWTH_2010_2014:.2f}% and a total difference of ${WEST_DIFF_2010_2014}.')
print(f'Their first major spike in spending began in 2015 and continued to grow at an elevated rate, totaling in a {WEST_PERCENT_DIFF_2014_2017:.2f}% increase from 2014 to 2017 and a total difference of ${WEST_DIFF_2014_2017}.')
print(f'Despite their spending growth slowing between 2018 and 2019, they still maintained a mean growth rate of {WEST_MEAN_GROWTH_2018_2020:.2f}% between 2018 and 2020, resulting in an overall {WEST_PERCENT_DIFF_2018_2020:.2f}% increase and a difference of ${WEST_DIFF_2018_2020}.')

Between 2010 and 2014, the West had minimal growth in expenditures. This resulted in a mean growth rate of 4.03% and a total difference of $90167000.0.
Their first major spike in spending began in 2015 and continued to grow at an elevated rate, totaling in a 137.92% increase from 2014 to 2017 and a total difference of $900016000.0.
Despite their spending growth slowing between 2018 and 2019, they still maintained a mean growth rate of 11.37% between 2018 and 2020, resulting in an overall 25.73% increase and a difference of $434278000.0.


### Expenditure Visualizations

In [23]:
tech_vocational_line_charts_combined = utils.create_combined_figure(national_totals_line,
                                                                    regionwise_totals_line,
                                                                    title= 'National vs Regionwise Tech & Vocational Education Trends',
                                                                    subplot_titles=('National', 'Regional')
                                                                    )

tech_vocational_line_charts_combined.show()

### Notes:

**National Expenditures Over Time:**  
- There is a steady positive growth in Tech and Vocational expenditures from 2010 to 2014.
- In 2014, there is a sharp increase in expenditures ending in 2016.
- In 2016, expenditures appear to steady.
- Positive growth is visible throughout. 
- Difference between 2014 & 2016: $ 4,350,353,000  
- Percent difference between 2014 & 2016: 82.64% 

**Regional Expenditures Over Time:**  

**Midwest**  
- The Midwest had minimal growth between 2010 and 2015, resulting in a 14.83% increase and an average growth rate of 3.04%. 
    - Total Difference: $132.69M  

- 2016 saw a 88.14% increase from 2015.

- 2016 to 2019 saw a mean growth rate of 32.87%, resulting in a $959.98M increase.  
- 2020 saw a -2.24% decrease from 2019, likely due to impacts from the COVID-19 pandemic.

**Northeast**  
- The Northeast had minimal growth between 2010 and 2013, resulting in a mean growth rate of 0.71%.  
    - Total difference: $23.14M  

- They saw their first spike in expenditures in 2014, up 20% from 2013. 
- Between 2013 and 2015, their mean growth rate increased to 6.58%.
- Despite their spending growth slowing after this initial spike, they maintained a fairly constant rate from 2015 to 2020, with a mean growth rate of 11.03%
    - Total Difference: $1.16B

**South**  
- The South saw its greatest spike in spending betweeen 2014 and 2016, resulting in a 99.71% increase.  
    - Total Difference: $2,33B  

- Their growth rate in 2017 slowed to 3.59% from 2016. 
- Despite the decreased growth rate in 2017, they had another spike in 2018, resulting in a 17.48% increase.
- 2017 to 2020 saw a mean growth rate of 10.09%, resulting in a $1.99B increase.
- The South has the most impact to the national trend between 2010 and 2020, which can be seen in the side by side chart comparison above. 

**West**  
- Between 2010 and 2014, the West had minimal growth in expenditures. This resulted in a mean growth rate of 4.03%. 
    - Total Difference: $90.17M  

- Their first major spike in spending began in 2015 and continued at an elevated rate, totaling in a 137.92% increase from 2014 to 2017. 
    - Total Difference: $9M  

- Despite their spending growth slowing between 2018 and 2019, they still maintained a mean growth rate of 11.37% between 2018 and 2020, resulting in an overall 25.73% increase.
    - Total Difference: $434.28M

#### Noteworthy Event in 2015:

The **Every Student Succeeds Act (ESSA)**, enacted in 2015, marked the transition from the **No Child Left Behind Act (NCLB)** and likely impacted educational spending trends. Both **NCLB** and **ESSA** were centered on standards-based education reform, emphasizing accountability and federal funding distribution across various educational domains. However, their primary focus wasn't explicitly geared toward technology and vocational education allocations. Despite lacking specific dedicated funding, their broader influence on education funding, accountability measures, and state discretion might have indirectly affected these areas.

In [24]:
regionwise_tech_vocational_expenditures.sort_values(by=['region', 'expenditure_title', 'year'], inplace=True)

# Calculate the year-to-year difference and growth rate by region and expenditure title
regionwise_tech_vocational_expenditures['yearly_difference'] = regionwise_tech_vocational_expenditures.groupby(['region', 'expenditure_title'])['amount'].diff()
regionwise_tech_vocational_expenditures['growth_rate'] = regionwise_tech_vocational_expenditures.groupby(['region', 'expenditure_title'])['amount'].pct_change() * 100

In [25]:
MIDWEST_TEACHER_SALARIES_MEAN_GROWTH = utils.calculate_mean_growth_rate(regionwise_tech_vocational_expenditures,
                                                                        2010, 2020,
                                                                        expenditure_title= 'teacher_salaries_vocational_education',
                                                                        region= 'Midwest')

print(f'Teacher Salaries - Vocational Education Mean Growth Rate 2010 to 2020: {MIDWEST_TEACHER_SALARIES_MEAN_GROWTH:.2f}%')

Teacher Salaries - Vocational Education Mean Growth Rate 2010 to 2020: 2.54%


In [28]:
regionwise_expenditure_totals_line = utils.make_line_plot_grid(regionwise_tech_vocational_expenditures,
                                                  x= 'year',
                                                  y= 'amount',
                                                  color= 'expenditure_title',
                                                  facet_col= 'region',
                                                  facet_col_wrap= 2,
                                                  title= 'Tech and Vocational Education Expenditure Trends by Region & Expenditure Title',
                                                  hover_data= ['yearly_difference', 'growth_rate']
                                                  )

regionwise_expenditure_totals_line.show()

#### Notes:

`tech_related_supplies_services` and `tech_related_equipment`
- First introduced in 2015
    - 2015 was also the end of the *No Child Left Behind Act (NCLB)* and the start of the *Every Student Succeeds Act (ESSA)*. This shift in policy could be an indicator of why these additional expenditure titles were introduced. 

**Midwest**
- `teacher_salaries_vocational_education` 
    - Remains quite stable between 2010 and 2020. Hovering close around the $1 Billion line.
    - Remains the leading expense between 2015 and 2017.

- `tech_related_supplies_services`
    - Has a steep climb from 2015 to 2016, after which point, spending becomes more steady, maintaining a consistent growth trend until 2019.
    - 2019 marks the year when spending surpasses `teacher_salaries_vocational_education` in total spending.

- `tech_related_equipment`
    - Has a steep climb from 2015 to 2016, after which point, spending becomes more steady. 
    - Spending increases at a constant rate beginning in 2017 until 2019.
    - 2020 indicates a decline in spending. 

In [27]:
regionwise_totals_line = utils.make_line_plot_grid(regionwise_expenditure_totals,
                                                  x= 'year',
                                                  y= 'growth_rate',
                                                  color= 'region',
                                                  facet_col= 'region',
                                                  facet_col_wrap= 2,
                                                  title= 'Tech and Vocational Education Expenditure Growth by Region Over Time'
                                                  )

regionwise_totals_line.update_traces(
    hovertemplate="<br>".join([
        "Year: %{x}",
        "Growth Rate: %{y:.2f}%",
        "Yearly Difference: %{customdata[0]:.2f}"
    ]),
    customdata=regionwise_expenditure_totals[['yearly_difference']].values
)

regionwise_totals_line.show()